In [ ]:
# Open notebook with jupyter notebook --port=8889 --no-browser

In [1]:
# modules 
import collections as cl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from scipy import stats
import math
import easygui
import tkinter as tk
from matplotlib.figure import Figure  #plotting fiure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg #Plotting tools
from tkinter import ttk

In [34]:
def app():
    # Select path for each file
#     path_cap_riviere = easygui.fileopenbox()  #P
#     path_cap_ZH = easygui.fileopenbox()  #T
#     path_etalonnage_capteur_riv = easygui.fileopenbox() #P508

    # Hardcoded path while testing
    path_cap_riviere = "../MOLONARI_1D_RESOURCES/sampling_points/Point034/point034_P_measures.csv"
    path_cap_ZH = "../MOLONARI_1D_RESOURCES/sampling_points/Point034/point034_T_measures.csv"
    path_etalonnage_capteur_riv = "../MOLONARI_1D_RESOURCES/configuration/pressure_sensors/P508.csv"

    #Opening files as DataFrames
    capteur_riviere = pd.read_csv(path_cap_riviere,header=1)
    capteur_ZH = pd.read_csv(path_cap_ZH,header=1)
    etalonnage_capteur_riv = pd.read_csv(path_etalonnage_capteur_riv)
    
    # Set seed for reproducibility
    np.random.seed(0)
    
    # Drop last columns. Not valuable data.
    capteur_riviere.drop(labels = list(capteur_riviere.columns)[4:],axis=1,inplace=True)
    capteur_ZH.drop(labels = list(capteur_ZH.columns)[6:],axis=1,inplace=True)

    # Rename columns as requested
    capteur_riviere.rename(columns={'#': '#', list(capteur_riviere.columns)[1]: 'dates',list(capteur_riviere.columns)[2]: 'tension_V',
                                    list(capteur_riviere.columns)[3]:'temperature_stream_C'}, inplace =  True)
    capteur_ZH.rename(columns={'#': '#', list(capteur_ZH.columns)[1]: 'dates', list(capteur_ZH.columns)[2]: 'temperature_depth_1_C',
                               list(capteur_ZH.columns)[3]:'temperature_depth_2_C', list(capteur_ZH.columns)[4]:'temperature_depth_3_C',
                               list(capteur_ZH.columns)[5]:'temperature_depth_4_C'}, inplace =  True)
    
    #Getting the charge difference in metters following "charge_m = (tens_V-temp_C·dU/dT-Intercept)/(dU/dH)"
    intercept = float(etalonnage_capteur_riv.iloc[2][list(etalonnage_capteur_riv.columns)[-1]])
    dUdH = float(etalonnage_capteur_riv.iloc[3][list(etalonnage_capteur_riv.columns)[-1]])
    dUdT = float(etalonnage_capteur_riv.iloc[4][list(etalonnage_capteur_riv.columns)[-1]])
    capteur_riviere["charge_m"] = (capteur_riviere["tension_V"]-capteur_riviere["temperature_stream_C"]*dUdT-intercept)/dUdH
    
    #converting dates in datetime64
    capteur_riviere['dates'] = pd.to_datetime(capteur_riviere['dates'], infer_datetime_format=True)
    capteur_ZH['dates'] = pd.to_datetime(capteur_ZH['dates'], infer_datetime_format=True)
    
    def verify_days(sensor):
        # Shows an histogram to verify that the quantity of data for each day makes sense (days must be between 1 and 31)
        # sensor : sensor DataFrame with a column named "dates" and type "datetime64"
        # Ex. : verify_days(capteur_riviere)
        days = sensor["dates"].dt.day
        plt.hist(days,range=[1,32],bins=31)
    
    def box_plot(sensor):
        # Shows a box plot for each of the sensor variables
        # sensor : sensor DataFrame with data of the variables from the third to the last column
        # Ex. : box_plot(capteur_ZH)
        fig, axs = plt.subplots(1, len(list(sensor.columns)[2:]),figsize=(20,5))
        for i in range(len(list(sensor.columns)[2:])):
            sensor.boxplot(column = list(sensor.columns)[2+i], ax=axs[i])
        plt.show()
     
    def histogram(sensor):
        # Shows an histogram for each of the sensor variables
        # sensor : sensor DataFrame with data of the variables from the third to the last column
        # Ex. : histogram(capteur_ZH)
        sensor[list(sensor.columns)[2:]].hist(bins=100,figsize=(20,4),layout=(1,len(list(sensor.columns)[2:])))
        plt.subplots_adjust(wspace=0.2,hspace=0.2)
        plt.show()
    
    def scatterplot(var1, var2):
        # Shows a scatterplot relating var1 and var2
        # var1, var2 : each one is a Series with the data of the variable.
        # Ex. : scatterplot(capteur_riviere["temperature_stream_C"], capteur_ZH['temperature_depth_3_C'])
        df = pd.concat([var1, var2], axis=1)
        df.plot.scatter(x = var1.name, y = var2.name)
        plt.subplots_adjust(wspace=0.2,hspace=0.2)
        plt.show()
    
    def big_scatterplot(sensor1, sensor2):
        # Shows a big scatterplot where all the variables of both sensors are related
        # sensor1, sensor2 : sensors DataFrame with data of the variables from the third to the last column
        # Ex. : big_scatterplot(capteur_riviere,capteur_ZH)
        var_number = 7
        fig_size = 24
        s1 = sensor1[list(sensor1.columns)[2:]].copy()
        s2 = sensor2[list(sensor2.columns)[2:]].copy()
        df = pd.concat([s1,s2], axis=1)
        fig, axs = plt.subplots(var_number, var_number,figsize=(fig_size,fig_size))
        for i in range(var_number):
            for j in range(var_number):
                df.plot.scatter(x = list(df.columns)[i], y = list(df.columns)[j],ax=axs[i,j])
        plt.subplots_adjust(wspace=0.32,hspace=0.32)
        plt.show()

    def get_df(capteur_riviere, capteur_ZH):
        s1 = capteur_riviere[list(capteur_riviere.columns)[3:]].copy()
        s2 = capteur_ZH[list(capteur_ZH.columns)[1:]].copy()
        df = pd.concat([s2,s1], axis=1)
        return df
        
    def remove_outlier_col_iqr(df_in, col_name):
        q1 = df_in[col_name].quantile(0.25)
        q3 = df_in[col_name].quantile(0.75)
        iqr = q3-q1 #Interquartile range
        fence_low  = q1-1.5*iqr
        fence_high = q3+1.5*iqr
        df_out = df_in.loc[(df_in[col_name] > fence_low) & (df_in[col_name] < fence_high)]
        return df_out
    
    def remove_outlier_df_iqr(cap_riv, cap_ZH):
        df = get_df(cap_riv, cap_ZH)
        df_or = df.copy()
#         Just to have the same result than Felix since he didn't use voltage and charge        
#         for i in range(1,len(list(df.columns))): 
#             if (i!=5 and i!=7): 
#                 df = remove_outlier_col_iqr(df,list(df.columns)[i])

        for i in range(1,len(list(df.columns))): 
            df = remove_outlier_col_iqr(df,list(df.columns)[i])
            
        return df, df_or
    
    def remove_outlier_z(cap_riv, cap_ZH):
        df = get_df(cap_riv, cap_ZH)
        df_or = df.copy()
        df.dropna(inplace = True)
        df = df[(np.abs(stats.zscore(df.iloc[:,1:])) < 3).all(axis=1)]
#         df = df[(np.abs(stats.zscore(df.iloc[:,[1,2,3,4,6]])) < 3).all(axis=1)] # Just to compare with Felix's code
        return df, df_or
                
    def plot_outliers(df, df_or):
        df1 = df_or.copy()
        df1.dropna(inplace = True)
        result = pd.concat([df_or, df], axis=1)
        df1['outliers'] = result[list(df1.columns)[1]].iloc[:,1]
        df1['outliers'][np.isnan(df1['outliers'])] = True
        df1['outliers'][df1['outliers'] != True] = False
        fig, axs = plt.subplots(4, 2,figsize=(10,20))
        first = df1["dates"].iloc[0]
        last = df1["dates"].iloc[-1]
        for i in range(1,len(list(df1.columns))-1):
            a = math.floor((i-1)/2)
            b = (i-1)%2
            df1[df1['outliers'] == False].plot(x='dates',y=list(df1.columns)[i],ax = axs[a,b])
            df1[df1['outliers'] == True].plot.scatter(x='dates',y=list(df1.columns)[i],c = 'r',s = 3,ax = axs[a,b])
            axs[a,b].tick_params(labelrotation=30)
            axs[a,b].set_xlim([datetime.date(first.year, first.month, first.day), datetime.date(last.year, last.month, last.day+1)])

        plt.subplots_adjust(wspace=0.3,hspace=0.5)
        plt.show()
        
    
        
    
        
#     df_iqr, df_or_iqr = remove_outlier_df_iqr(capteur_riviere, capteur_ZH)
#     plot_outliers(df_iqr, df_or_iqr)
#     df_z, df_or_z = remove_outlier_z(capteur_riviere, capteur_ZH)
#     plot_outliers(df_z, df_or_z)
    
app()

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 13: invalid continuation byte

In [39]:
np.random.seed(0)
plot1 = Figure(figsize=(5,5), dpi=100, edgecolor="black") #Here is the plotting
ax1=plot1.add_subplot(111)
textFont_L = ("Seoge UI",14)
textFont_S = ("Seoge UI",10)

class var_dic:
    
    def __init__(self,name,value,dics_list): 
        self.name = name
        self.value = value
        self.dics_list = dics_list
        if "t1" in self.dics_list:
            verifyDays_d[self.name] = self.value
        if "t2" in self.dics_list:
            plots_d[self.name] = self.value
        
    def set_value(self,value):        
        self.value = value
        if "t1" in self.dics_list:
            verifyDays_d[self.name] = self.value
        if "t2" in self.dics_list:
            plots_d[self.name] = self.value
    


class MOLONARI_GUI(tk.Tk): #This is our app
    def __init__(self, *args, **kwargs): #Starts what we need
        tk.Tk.__init__(self, *args, **kwargs)
        tk.Tk.title(self, "MOLONARI - Traitement des données")

        container1=tk.Frame(self) #The containers will have the frames inside them
        container1.grid(row=0, column=0)
        
        container2=tk.Frame(self)
        container2.grid(row=0, column=2, rowspan=2)        
        
        frame1=frameOptions(container1,self) #assigns a frame to its container
        frame1.grid(row=0, column=0, sticky="nsew")
        
        frame2 = framePlots(container2,self)
        frame2.grid(row=0, column=2,sticky="nsew")

    
class frameOptions(tk.Frame): #Frame with buttons inside
    def __init__(self, parent, controller):
        global path_cap_riviere, path_cap_ZH, path_etalonnage_capteur_riv
        global dataFrames
        global verifyDays_d, plots_d
        
        verifyDays_d = {}
        plots_d = {}
#         outliers_d = {"IQR Method": [df_iqr, df_or_iqr],"Z Method": [df_z, df_or_z]}
        
        
        # Just to test
        path_cap_riviere = "../MOLONARI_1D_RESOURCES/sampling_points/Point034/point034_P_measures.csv"
        path_cap_ZH = "../MOLONARI_1D_RESOURCES/sampling_points/Point034/point034_T_measures.csv"
        path_etalonnage_capteur_riv = "../MOLONARI_1D_RESOURCES/configuration/pressure_sensors/P508.csv"
        
        # Uncomment when finished
#         path_cap_riviere = ""
#         path_cap_ZH = ""
#         path_etalonnage_capteur_riv = ""

        WH1, HT1 = 300, 500  #WIDTH, HEIGHT canvasBkg1
        WH2, HT2 = 500, HT1*0.8
        WH3, HT3 = WH2, HT1-HT2
        spacing = 0.1 #Vertical spacing
        first = 0.05 #first element vertical position
        
        frameOptionsBG1 = "#FBFCF5"
        buttonsBG1 = "#E6E7E0"
        activebuttonsBG1 = "#F3F4EF"
        
        
        buttonsBG2 = "#E6E7E0"
        activebuttonsBG2 = "#F3F4EF"
        
        frameOptionsBG3 = "#FEFFFC"
        
        tk.Frame.__init__(self,parent)
        tk.Frame.config(self, background="red",bd=30) #Initial configuration
        
        canvasBkg1 = tk.Canvas(width=WH1,height=HT1,bg=frameOptionsBG1) #Creates options canvas
        canvasBkg1.grid(row=0,column=0, rowspan=2)
        
        canvasBkg2 = tk.Canvas(width=WH2,height=HT2,bg=frameOptionsBG3) #Creates selection canvas
        canvasBkg2.grid(row=0,column=1)
        
        canvasBkg3 = tk.Canvas(width=WH3,height=HT3,bg=frameOptionsBG3) #Creates text canvas
        canvasBkg3.grid(row=1,column=1)
        
        def getPath(entryPath,var):
            entryPath.configure(state="normal")
            var = easygui.fileopenbox()
            if (var[-4:]!=".csv"):
                tk.messagebox.showwarning(title="Error", message="Wrong file extension. Expected .csv file")
                entryPath.delete(0,tk.END)
                entryPath.configure(state="disabled")
                return ""
            entryPath.insert(0,var)
            entryPath.configure(state="disabled")
            return var
            
        def dotsButton1_f(entryPath): 
            global path_cap_riviere
            path_cap_riviere = getPath(entryPath, path_cap_riviere)
                
        def dotsButton2_f(entryPath):
            global path_cap_ZH
            path_cap_ZH = getPath(entryPath, path_cap_ZH)
            
        def dotsButton3_f(entryPath):
            global path_etalonnage_capteur_riv
            path_etalonnage_capteur_riv = getPath(entryPath, path_etalonnage_capteur_riv)
            
        def uploadButton_f():
            global path_cap_riviere, path_cap_ZH, path_etalonnage_capteur_riv
            global capteur_riviere, capteur_ZH, etalonnage_capteur_riv
            global verifyDays_list
            #Upload files
            try:
                capteur_riviere = pd.read_csv(path_cap_riviere,header=1)
                capteur_ZH = pd.read_csv(path_cap_ZH,header=1)
                etalonnage_capteur_riv = pd.read_csv(path_etalonnage_capteur_riv)
                
            except:
                tk.messagebox.showwarning(title="Error", message="Something went wrong uploading the files. Please try again or use another file")
            
            # Drop last 3 columns. Not valuable data.
            try:
                capteur_riviere.drop(labels = list(capteur_riviere.columns)[4:],axis=1,inplace=True)
                capteur_ZH.drop(labels = list(capteur_ZH.columns)[6:],axis=1,inplace=True)
            except:
                tk.messagebox.showwarning(title="Error", message="Something went wrong. Make sure the last 3 columns are not valuable data")
            
            # Rename columns as requested
            try:
                capteur_riviere.rename(columns={'#': '#', list(capteur_riviere.columns)[1]: 'dates',list(capteur_riviere.columns)[2]: 'tension_V',
                                                list(capteur_riviere.columns)[3]:'temperature_stream_C'}, inplace =  True)
                capteur_ZH.rename(columns={'#': '#', list(capteur_ZH.columns)[1]: 'dates', list(capteur_ZH.columns)[2]: 'temperature_depth_1_C',
                                           list(capteur_ZH.columns)[3]:'temperature_depth_2_C', list(capteur_ZH.columns)[4]:'temperature_depth_3_C',
                                           list(capteur_ZH.columns)[5]:'temperature_depth_4_C'}, inplace =  True)
            except:
                tk.messagebox.showwarning(title="Error", message="Something went wrong. Make sure column formats are as expected")
            
            #Getting the charge difference in metters following "charge_m = (tens_V-temp_C·dU/dT-Intercept)/(dU/dH)"
            try:
                intercept = float(etalonnage_capteur_riv.iloc[2][list(etalonnage_capteur_riv.columns)[-1]])
                dUdH = float(etalonnage_capteur_riv.iloc[3][list(etalonnage_capteur_riv.columns)[-1]])
                dUdT = float(etalonnage_capteur_riv.iloc[4][list(etalonnage_capteur_riv.columns)[-1]])
                capteur_riviere["charge_m"] = (capteur_riviere["tension_V"]-capteur_riviere["temperature_stream_C"]*dUdT-intercept)/dUdH
            except:
                tk.messagebox.showwarning(title="Error", message="Something went wrong getting the charge difference in metters.")
                
            #converting dates in datetime64
            try:
                capteur_riviere['dates'] = pd.to_datetime(capteur_riviere['dates'], infer_datetime_format=True)
                capteur_ZH['dates'] = pd.to_datetime(capteur_ZH['dates'], infer_datetime_format=True)
            except:
                tk.messagebox.showwarning(title="Error", message="Something went wrong converting dates in datetime")
            
            
            cap_riv_dic = var_dic("Capteur rivière",capteur_riviere,["t1", "t2"])
            cap_ZH_dic = var_dic("Capteur ZH", capteur_ZH, ["t1","t2"])
                    
            tk.messagebox.showinfo(title="Uploaded", message="Files have been succesfully uploaded")
            
        def selectData_f():
            canvasBkg2.delete("all")
            canvasBkg3.delete("all")
            
            capRivText = tk.Label(text="Capteur rivière",font=textFont_S, bg=frameOptionsBG3)
            capRivText_w = canvasBkg2.create_window(WH2*0.11, HT2*(first+spacing*1), anchor="w", window=capRivText)
            
            dotsButton1 = tk.Button(text="...", bd=0, cursor="hand2",bg=buttonsBG2,activebackground=activebuttonsBG2, command=lambda:dotsButton1_f(entryPath1))
            dotsButton1_w = canvasBkg2.create_window(WH2*0.1, HT2*(first+spacing*1.5),anchor="e",window=dotsButton1)
            
            entryPath1 = tk.Entry(textvariable=path_cap_riviere,bd=0,width=50,state="disabled")
            entryPath1_w = canvasBkg2.create_window(WH2*0.11, HT2*(first+spacing*1.5),anchor="w",window=entryPath1)	
            
            capZHText = tk.Label(text="Capteur zone hyporhéique",font=textFont_S, bg=frameOptionsBG3)
            capZHText_w = canvasBkg2.create_window(WH2*0.11, HT2*(first+spacing*2), anchor="w", window=capZHText)
            
            dotsButton2 = tk.Button(text="...", bd=0, cursor="hand2",bg=buttonsBG2, activebackground=activebuttonsBG2, command=lambda:dotsButton2_f(entryPath2))
            dotsButton2_w = canvasBkg2.create_window(WH2*0.1, HT2*(first+spacing*2.5),anchor="e",window=dotsButton2)
            
            entryPath2 = tk.Entry(textvariable=path_cap_ZH,bd=0,width=50,state="disabled")
            entryPath2_w = canvasBkg2.create_window(WH2*0.11, HT2*(first+spacing*2.5),anchor="w",window=entryPath2)
            
            etalText = tk.Label(text="Étalonnage capteur rivière",font=textFont_S, bg=frameOptionsBG3)
            etalText_w = canvasBkg2.create_window(WH2*0.11, HT2*(first+spacing*3), anchor="w", window=etalText)
            
            dotsButton3 = tk.Button(text="...", bd=0, cursor="hand2",bg=buttonsBG2, activebackground=activebuttonsBG2, command=lambda:dotsButton3_f(entryPath3))
            dotsButton3_w = canvasBkg2.create_window(WH2*0.1, HT2*(first+spacing*3.5),anchor="e",window=dotsButton3)
            
            entryPath3 = tk.Entry(textvariable=path_etalonnage_capteur_riv,bd=0,width=50,state="disabled")
            entryPath3_w = canvasBkg2.create_window(WH2*0.11, HT2*(first+spacing*3.5),anchor="w",window=entryPath3)
            
            uploadButton = tk.Button(text="Upload files", bd=0, cursor="hand2",bg=buttonsBG2, activebackground=activebuttonsBG2, command=uploadButton_f)
            uploadButton_w = canvasBkg2.create_window(WH2*0.5, HT2*(first+spacing*4.5),anchor="center",window=uploadButton)
            
            text = tk.Label(text="Find the the data sets paths\nin .csv format",font=textFont_L, bg=frameOptionsBG3)
            canvasText_w = canvasBkg3.create_window(WH3*0.5, HT3*first, anchor="n", window=text)
                  
        def get_sensorDF(box, dictionary):
            try:
                sensorName = box.get(box.curselection())
            except tk.TclError:
                tk.messagebox.showwarning(title="Error", message="You have not selected any sensor")
                return 0
            sensor = dictionary[sensorName]
            return sensor
            
        def daysHistogram_f(verifyDaysBox):
            
            sensor = get_sensorDF(verifyDaysBox, verifyDays_d)
            # Shows an histogram to verify that the quantity of data for each day makes sense (days must be between 1 and 31)
            # sensor : sensor DataFrame with a column named "dates" and type "datetime64"
            days = sensor["dates"].dt.day            
            plot1.clf()
            axs = plot1.subplots(nrows=1, ncols=1)
            axs.hist(days,range=[1,32],bins=31)
            canvas1.draw()
        
        def verifyDays_f():
            
            verifyDays_var = tuple(verifyDays_d.keys())
            verifyDays_list = tk.StringVar(value=verifyDays_var)
            
            canvasBkg2.delete("all")
            canvasBkg3.delete("all")    
            
            verifyDaysBox = tk.Listbox(listvariable=verifyDays_list, height=len(verifyDays_var))
            canvasVerifyBox_w = canvasBkg2.create_window(WH2*0.5, HT2*first, anchor="n", window=verifyDaysBox)
            
            daysHistogram = tk.Button(text="Days Histogram", bd=0, cursor="hand2",bg=buttonsBG2,activebackground=activebuttonsBG2, command=lambda:daysHistogram_f(verifyDaysBox))
            daysHistogram_w = canvasBkg2.create_window(WH2*0.5, HT2*(first+spacing*1.5),anchor="n",window=daysHistogram)
            
            text1 = tk.Label(text="Use this to verify that the sensor \n recorded data uniformly throughout the days",font=textFont_L, bg=frameOptionsBG3)
            canvasText1_w = canvasBkg3.create_window(WH3*0.5, HT3*first, anchor="n", window=text1)
            
            text2 = tk.Label(text="Distribution should be between 1 and 31.\nIf something is not as expected, upload files once again",font=textFont_S, bg=frameOptionsBG3)
            canvasText2_w = canvasBkg3.create_window(WH3*0.5, HT3*(first+spacing*5.5), anchor="n", window=text2)
            
        def changeSensor_f(button, varBox, sensorBox):
            sensorBox.config(state="normal")
            varBox.config(listvariable=tk.StringVar(''), height=0)
            button.config(text="Select sensor", command=lambda:selectSensor_f(button, varBox, sensorBox))
            
        def selectSensor_f(button, varBox, sensorBox):
            sensor = get_sensorDF(sensorBox, plots_d)
            
            list_tuple = tuple(list(sensor.columns))
            list_StringVar = tk.StringVar(value=list_tuple)
            varBox.config(listvariable=list_StringVar, height=len(list_tuple))
            
            sensorBox.config(state="disabled")
            button.config(text="Change sensor", command=lambda:changeSensor_f(button,varBox, sensorBox))
            
            selectVarButton = tk.Button(text="Select variable and plot",bd=0, cursor="hand2",bg=buttonsBG1,activebackground=activebuttonsBG1, command=lambda:selectSensor_f(selectSensorButton, varBox, sensorBox))
            canvasSelectSensor_w = canvasBkg2.create_window(WH2*0.2, HT2*first*5.5, anchor="n", window=selectSensorButton)
            
            
        def boxPlot_f():
            canvasBkg2.delete("all")
            canvasBkg3.delete("all")         
            
            plots_var = tuple(plots_d.keys())
            plots_list = tk.StringVar(value=plots_var)
            
            sensorBox = tk.Listbox(listvariable=plots_list, height=len(plots_var))
            canvasSensorBox_w = canvasBkg2.create_window(WH2*0.2, HT2*first, anchor="n", window=sensorBox)
            
            varBox = tk.Listbox(height=1)
            canvasVarBox_w = canvasBkg2.create_window(WH2*0.6, HT2*first, anchor="n", window=varBox)
            
            selectSensorButton = tk.Button(text="Select sensor",bd=0, cursor="hand2",bg=buttonsBG1,activebackground=activebuttonsBG1, command=lambda:selectSensor_f(selectSensorButton, varBox, sensorBox))
            canvasSelectSensor_w = canvasBkg2.create_window(WH2*0.2, HT2*first*5.5, anchor="n", window=selectSensorButton)
        
        titleLabel = tk.Label(text="OPTIONS", font=textFont_L, bg=frameOptionsBG1)
        canvasTitle_w = canvasBkg1.create_window(WH1*0.5, HT1*first, anchor="center", window=titleLabel)
        
        selectDataButton = tk.Button(text="Select data",bd=0, cursor="hand2",bg=buttonsBG1,activebackground=activebuttonsBG1, command=selectData_f)
        selectData_w = canvasBkg1.create_window(WH1*0.5,HT1*(first+spacing*1.5),anchor="center",window=selectDataButton)

        verifyDaysButton = tk.Button(text="Verify days",bd=0, cursor="hand2",bg=buttonsBG1,activebackground=activebuttonsBG1, command=verifyDays_f)
        verifyDays_w = canvasBkg1.create_window(WH1*0.5,HT1*(first+spacing*2.5),anchor="center",window=verifyDaysButton)
        
        boxPlotButton = tk.Button(text="Box plot",bd=0, cursor="hand2",bg=buttonsBG1,activebackground=activebuttonsBG1, command=boxPlot_f)
        boxPlot_w = canvasBkg1.create_window(WH1*0.5,HT1*(first+spacing*3.5),anchor="center",window=boxPlotButton)
        
        histogramButton = tk.Button(text="Histogram",bd=0, cursor="hand2",bg=buttonsBG1,activebackground=activebuttonsBG1)
        histogram_w = canvasBkg1.create_window(WH1*0.5,HT1*(first+spacing*4.5),anchor="center",window=histogramButton)
        
        scatterPlotButton = tk.Button(text="Scatter plot",bd=0, cursor="hand2",bg=buttonsBG1,activebackground=activebuttonsBG1)
        scatterPlot_w = canvasBkg1.create_window(WH1*0.5,HT1*(first+spacing*5.5),anchor="center",window=scatterPlotButton)
        
        iqrButton = tk.Button(text="IQR Method",bd=0, cursor="hand2",bg=buttonsBG1,activebackground=activebuttonsBG1)
        iqr_w = canvasBkg1.create_window(WH1*0.5,HT1*(first+spacing*6.5),anchor="center",window=iqrButton)
        
        zButton = tk.Button(text="Z Method",bd=0, cursor="hand2",bg=buttonsBG1,activebackground=activebuttonsBG1)
        z_w = canvasBkg1.create_window(WH1*0.5,HT1*(first+spacing*7.5),anchor="center",window=zButton)
        
        plotOutliersButton = tk.Button(text="Plot outliers",bd=0, cursor="hand2",bg=buttonsBG1,activebackground=activebuttonsBG1)
        plotOutliers_w = canvasBkg1.create_window(WH1*0.5,HT1*(first+spacing*8.5),anchor="center",window=plotOutliersButton)
        
     
             
        
class framePlots(tk.Frame): 
    def __init__(self, parent, controler):
        global canvas1
        tk.Frame.__init__(self,parent)
        tk.Frame.config(self, background="#95ab3d") #Initial configuration
        
        canvas1 = FigureCanvasTkAgg(plot1, self) #Widget who has plot1 inside
        canvas1.get_tk_widget().grid(row=0,column=4)
        t = np.linspace(0,10,201)
        x = np.sin(t)
        ax1.plot(t,x)
        
app=MOLONARI_GUI() #Starts GUI
app.mainloop() #GUI Loop

In [50]:
app=MOLONARI_GUI() #Starts GUI
app.mainloop() #GUI Loop

In [ ]:
# Example code to redraw the plot
global canvas1
plot1.clf()
axs = plot1.subplots(nrows=3, ncols=3)
t = np.linspace(0,10,201)
x = np.cos(t)
axs[2,2].plot(t,x)
canvas1.draw()

In [19]:
v1 = [132, 312]
v1_name = "numero 1"
v2 = 321
v2_name = "numero 2"
d = {v1_name : v1, v2_name : v2}
print(d[list(d.keys())[0]])


[132, 312]


In [27]:
class dictionary():
    def __init__(self):
        self.capteur_riviere = 1
        self.capteur_ZH = 2
        self.etalonnage_capteur_riv = 3
        self.dicti = {"Capteur rivière" : self.capteur_riviere, "Capteur zone hyporhéique" : self.capteur_ZH}
    
    def set_var(self,var,value):
        self.var = value


pr = dictionary()
print(pr.dicti["Capteur zone hyporhéique"])
pr.set_var("capteur_riviere",5)
print(pr.var)
print(pr.dicti["Capteur zone hyporhéique"])

2
5
2


In [20]:
path_cap_ZH = "../MOLONARI_1D_RESOURCES/sampling_points/Point046/point046_T_measures.csv"
capteur_ZH = pd.read_csv(path_cap_ZH)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 13: invalid continuation byte

In [77]:
print("a"=='a')

True
